In [3]:
import pandas as pd
slice1 = pd.read_csv('../datasets/slice1.csv', sep=";")
slice2 = pd.read_csv('../datasets/slice2.csv', sep=";")
reclamaciones = pd.read_csv('../datasets/reclamaciones.csv', sep = ";")
requerimientos = pd.read_csv('../datasets/requerimientos.csv', sep = ";")

/Users/ireneroma/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
user = 'C241446332'
slice1.ANYMES_CAL = pd.to_datetime(slice1.ANYMES_CAL, format = '%Y%m')
slice2.ANYMES_CAL = pd.to_datetime(slice2.ANYMES_CAL, format = '%Y%m')
slice1_user = slice1[slice1.ContratoCOD == str(user)]
slice2_user = slice2[slice2.ContratoCOD == str(user)]
user_consum = pd.concat([slice1_user, slice2_user])
user_recl = reclamaciones[reclamaciones.ContratoCOD == str(user)]
user_req = requerimientos[requerimientos.ContratoCOD == str(user)]

In [60]:
user_consum.head()

,DESC_MUNI,DES_DISTRIC,ANYMES_CAL,DESC_TIP_SUBM,CONSUM,USO,ContratoCOD
2644,SANT CLIMENT LLOB.,Sense Informar,2014-07-01,SUBMINISTRAMENT COMPTADOR GENERAL,20,DOMESTIC,C241446332
2645,SANT CLIMENT LLOB.,Sense Informar,2011-11-01,SUBMINISTRAMENT COMPTADOR GENERAL,17,DOMESTIC,C241446332
2646,SANT CLIMENT LLOB.,Sense Informar,2015-03-01,SUBMINISTRAMENT COMPTADOR GENERAL,20,DOMESTIC,C241446332
2647,SANT CLIMENT LLOB.,Sense Informar,2013-05-01,SUBMINISTRAMENT COMPTADOR GENERAL,17,DOMESTIC,C241446332
2648,SANT CLIMENT LLOB.,Sense Informar,2016-07-01,SUBMINISTRAMENT COMPTADOR GENERAL,5,DOMESTIC,C241446332


In [87]:
user_consum1 = pd.DataFrame()
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    return date.replace(day=d,month=m, year=y)
for i in range(0, len(user_consum.ANYMES_CAL)-1):
    if ((user_consum.ANYMES_CAL.iloc[i].year == 2011)|(user_consum.ANYMES_CAL.iloc[i].year == 2012))&(user_consum.CONSUM.iloc[i]>0):
        consum_mes = user_consum.CONSUM.iloc[i]/3
        dos_mes = pd.DataFrame({'DESC_MUNI':[user_consum.DESC_MUNI.iloc[i]],'DES_DISTRIC':[user_consum.DES_DISTRIC.iloc[i]],'ANYMES_CAL':[monthdelta(user_consum.ANYMES_CAL.iloc[i],-2)],'DESC_TIP_SUBM':[user_consum.DESC_TIP_SUBM.iloc[i]],'CONSUM':[consum_mes],'USO':[user_consum.USO.iloc[i]],'ContratoCOD':[user_consum.ContratoCOD.iloc[i]]})
        un_mes = pd.DataFrame({'DESC_MUNI':[user_consum.DESC_MUNI.iloc[i]],'DES_DISTRIC':[user_consum.DES_DISTRIC.iloc[i]],'ANYMES_CAL':[monthdelta(user_consum.ANYMES_CAL.iloc[i],-1)],'DESC_TIP_SUBM':[user_consum.DESC_TIP_SUBM.iloc[i]],'CONSUM':[consum_mes],'USO':[user_consum.USO.iloc[i]], 'ContratoCOD':[user_consum.ContratoCOD.iloc[i]]})
        zero_mes = pd.DataFrame({'DESC_MUNI':[user_consum.DESC_MUNI.iloc[i]],'DES_DISTRIC':[user_consum.DES_DISTRIC.iloc[i]],'ANYMES_CAL':[monthdelta(user_consum.ANYMES_CAL.iloc[i],0)],'DESC_TIP_SUBM':[user_consum.DESC_TIP_SUBM.iloc[i]],'CONSUM':[consum_mes],'USO':[user_consum.USO.iloc[i]], 'ContratoCOD':[user_consum.ContratoCOD.iloc[i]]})
        user_consum1 = pd.concat([user_consum1,dos_mes,un_mes,zero_mes])
    if ((user_consum.ANYMES_CAL.iloc[i].year == 2013)&(user_consum.CONSUM.iloc[i]>0)):
        consum_mes = user_consum.CONSUM.iloc[i]/2
        un_mes = pd.DataFrame({'DESC_MUNI':[user_consum.DESC_MUNI.iloc[i]],'DES_DISTRIC':[user_consum.DES_DISTRIC.iloc[i]],'ANYMES_CAL':[monthdelta(user_consum.ANYMES_CAL.iloc[i],-1)],'DESC_TIP_SUBM':[user_consum.DESC_TIP_SUBM.iloc[i]],'CONSUM':[consum_mes],'USO':[user_consum.USO.iloc[i]], 'ContratoCOD':[user_consum.ContratoCOD.iloc[i]]})
        zero_mes = pd.DataFrame({'DESC_MUNI':[user_consum.DESC_MUNI.iloc[i]],'DES_DISTRIC':[user_consum.DES_DISTRIC.iloc[i]],'ANYMES_CAL':[monthdelta(user_consum.ANYMES_CAL.iloc[i],0)],'DESC_TIP_SUBM':[user_consum.DESC_TIP_SUBM.iloc[i]],'CONSUM':[consum_mes],'USO':[user_consum.USO.iloc[i]], 'ContratoCOD':[user_consum.ContratoCOD.iloc[i]]})
        user_consum1 = pd.concat([user_consum1,un_mes,zero_mes])
       

In [89]:
import datetime
import numpy as np
date2 = datetime.datetime.strptime("2017-06-01", "%Y-%m-%d").date()
from sklearn.linear_model import LinearRegression
model = LinearRegression()
y = np.asarray(user_consum1['CONSUM'])
user_consum1['ANYMES_CAL']=user_consum1['ANYMES_CAL'].map(datetime.datetime.toordinal)
X = user_consum1[['ANYMES_CAL']]
model.fit(X, y)
model.score(X, y)
coefs = zip(model.coef_, X.columns)
result = model.predict(date2.toordinal())
print "sl = %.1f + " % model.intercept_ + \
" + ".join("%.1f %s" % coef for coef in coefs)

sl = -4603.6 + 0.0 ANYMES_CAL


In [93]:
user_recl.Fin = pd.to_datetime(user_recl.Fin, format = '%Y-%m-%d %H:%M:%S')
user_req.Fin = pd.to_datetime(user_req.Fin, format = '%Y-%m-%d %H:%M:%S')
if (user_recl.Fin.year == 2016):
    num_recl = len(user_recl[user_recl.Fin.year == 2016])
    if (user_req.Fin.year == 2016):
        num_req = len(user_req[user_req.Fin.year == 2016])

AttributeError: 'Series' object has no attribute 'year'

In [25]:
user_recl

,Unnamed: 0,IDContactoCod,TipoContacto,Tipo,Motivo,Estado,IDFincaCod,Municipio,ContratoCOD,Inicio,Fin
1,12605,A61745818,CE,Correo - Recibido,Reclamaci�n,Cerrado,B32992249,SANT CLIMENT DE LLOBREGAT,C241446332,2016-04-25 16:42:28,2016-04-25 16:46:44


In [26]:
user_req

,Unnamed: 0,Inicio,Fin,Municipio,Codpostal,Estado,Categoria,Asunto,Tema,Motivo,Uso,IDContactoCod,ContratoCOD
107079,1861649,2015-11-14 20:55:59,2015-11-15 11:46:04,SANT CLIMENT DE LLOBREGAT,8849.0,Cerrada,Reclamaci�n t�cnica,Escape,Cuarto de bateria,Otros,DOMESTIC,A257505242,C241446332


In [ ]:
month = datetime.timedelta(months=1)